In [9]:
import time
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [8]:

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])


trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

testset=datasets.MNIST('~/.pytorch/MNIST_data/',download=True,train=False,transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)



In [4]:

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(784, 256)
        self.output = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        # self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = x.view(-1, 28 * 28)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x
    
start=time.time()
if torch.cuda.is_available():
  print("device:cuda")
else:
  print("device:cpu")
model=Network()
print("model parameters",model)

device:cpu
model parameters Network(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
)


In [5]:
#training
learning_rate=0.001
epochs=31
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for e in range(epochs):
    correct=0
    for (images, labels) in (trainloader):
      
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        _,pred=torch.max(output.data,1)
      
        #total += labels.size(0)
        correct+=(pred==labels).sum() 
    if e%5==0:
      print(f"epoch: {e+1} , Training loss: {loss.item()},accuracy:{(correct/len(trainset))*100:.2f},%")

epoch: 1 , Training loss: 1.7004684209823608,accuracy:47.25,%
epoch: 6 , Training loss: 0.532974898815155,accuracy:85.01,%
epoch: 11 , Training loss: 0.48912814259529114,accuracy:87.90,%
epoch: 16 , Training loss: 0.41795673966407776,accuracy:89.03,%
epoch: 21 , Training loss: 0.33771437406539917,accuracy:89.74,%
epoch: 26 , Training loss: 0.3219013214111328,accuracy:90.21,%
epoch: 31 , Training loss: 0.24719008803367615,accuracy:90.53,%


In [6]:
with torch.no_grad():
    #dont calculate grad during testing
    correct = 0
    for (images, labels) in (testloader):
        output = model(images)
        _,pred=torch.max(output.data,1)
        correct+=(pred==labels).sum()
        end=time.time()
    print(f"accuracy:{(correct/len(testset))*100:.2f},%")
    print("time taken by cpu ",end-start)

accuracy:91.17,%
time taken by cpu  397.105770111084
